In [7]:
print(os.getcwd())
path = 'c:/Users/pipeg/Documents/GitHub/nba-team-creator/'
os.chdir(path)
os.getcwd()

c:\Users\pipeg\Documents\GitHub\nba-team-creator\optimization


'c:\\Users\\pipeg\\Documents\\GitHub\\nba-team-creator'

In [8]:
from pulp import *
import pandas as pd


In [9]:
players = pd.read_csv('out_data/players_model_data.csv')
players

,Player,Pos,Team,Season,PER,Unit,DRtg,Salary,Salary adjusted by inflation
0,Deni Avdija,SF,Washington Wizards,2020/21,7.6,2,113.0,4469160,4469160
1,Bradley Beal,SG,Washington Wizards,2020/21,22.7,1,115.0,28751774,28751774
2,Jordan Bell,C,Washington Wizards,2020/21,8.5,3,109.0,592368,592368
3,Jordan Bell,C,Golden State Warriors,2020/21,5.6,3,104.0,592368,592368
4,Davis Bertans,PF,Washington Wizards,2020/21,11.4,1,116.0,15000000,15000000
...,...,...,...,...,...,...,...,...,...
11422,Roshown McLeod,SF,Atlanta Hawks,1999/00,9.1,2,110.0,914760,1418907
11423,Dikembe Mutombo,C,Atlanta Hawks,1999/00,19.4,1,101.0,12820249,19885810
11424,Isaiah Rider,SG,Atlanta Hawks,1999/00,15.7,1,111.0,5410000,8391587
11425,Jason Terry,PG,Atlanta Hawks,1999/00,13.9,2,108.0,1468920,2278478


In [12]:
players.Unit = players.Unit.astype(str)

In [16]:
# Numerical Unit category to strings
players.Unit = players.Unit.astype(str)
# Get the dummies for Position and Unit (unit is a ordinal variable according to time played)
get_dummies = pd.get_dummies(players[['Pos', 'Unit']])
# Add the dummies to the dataset
players = pd.concat([players, get_dummies], axis = 1).drop(columns = ['Pos', 'Unit'])
players

,Player,Team,Season,PER,DRtg,Salary,Salary adjusted by inflation,Pos_C,Pos_PF,Pos_PG,Pos_SF,Pos_SG,Unit_1,Unit_2,Unit_3
0,Deni Avdija,Washington Wizards,2020/21,7.6,113.0,4469160,4469160,0,0,0,1,0,0,1,0
1,Bradley Beal,Washington Wizards,2020/21,22.7,115.0,28751774,28751774,0,0,0,0,1,1,0,0
2,Jordan Bell,Washington Wizards,2020/21,8.5,109.0,592368,592368,1,0,0,0,0,0,0,1
3,Jordan Bell,Golden State Warriors,2020/21,5.6,104.0,592368,592368,1,0,0,0,0,0,0,1
4,Davis Bertans,Washington Wizards,2020/21,11.4,116.0,15000000,15000000,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11422,Roshown McLeod,Atlanta Hawks,1999/00,9.1,110.0,914760,1418907,0,0,0,1,0,0,1,0
11423,Dikembe Mutombo,Atlanta Hawks,1999/00,19.4,101.0,12820249,19885810,1,0,0,0,0,1,0,0
11424,Isaiah Rider,Atlanta Hawks,1999/00,15.7,111.0,5410000,8391587,0,0,0,0,1,1,0,0
11425,Jason Terry,Atlanta Hawks,1999/00,13.9,108.0,1468920,2278478,0,0,1,0,0,0,1,0


Create Decision Variables: Every player

In [17]:
vars = []
for rownum, row in players.iterrows(): #For every row create a decision variable
    var = str('x' + str(rownum))
    var = pulp.LpVariable(str(var), lowBound = 0, upBound = 1, cat= 'Integer') # Decision problem is binary(0,1)
    vars.append(var)

In [28]:
# last player must be x
vars[-1]

x11426

In [34]:
len(vars) == len(players)

True

Defining the Optimization problem